[medium](https://medium.com/@jonathan_hui/rl-dqn-deep-q-network-e207751f7ae4) <br>
[ref](https://github.com/udacity/deep-reinforcement-learning/tree/master/dqn)

In [ ]:
%reload_ext autoreload
%autoreload 2
import torch
import torch.nn.functional as F 
import random
import numpy as np
from EXITrl.approx_v_base import ApproxVBase
from EXITrl.approx_policy_base import ApproxPolicyBase
from EXITrl.base import Base
from EXITrl.helpers import print_weight_size, copy_params, update_params, ExperienceReplay, convert_to_tensor
from EXITrl.nn_wrapper import NNWrapper
import gym

In [16]:
class QNetwork(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.linear1 = torch.nn.Linear(input_size, hidden_size)
        self.linear2 = torch.nn.Linear(hidden_size, hidden_size)
        self.linear3 = torch.nn.Linear(hidden_size, output_size)

    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        return self.linear3(x)

In [22]:
class DQN(Base):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.local_q_network = NNWrapper(
            QNetwork(self.num_state, 64, self.num_action),
            lr=self.alpha
        )
        self.target_q_network = NNWrapper(
            QNetwork(self.num_state, 64, self.num_action),
            lr=0 # use manual update
        )
        self.experience_replay = ExperienceReplay(num_experience=2048)
        
        self.epsilon = 1
        self.epsilon_decay = .995
        self.epsilon_end = 0.01
        self.num_step = 0
        
    def policy(self, state):
        return self.local_q_network.epsilon_greedy(state, self.epsilon)
    
    def learn(self, state, action, reward, next_state, done):
        # detach because we only backprop local network and update target network weight manually
        targets_next_Q = self.target_q_network.forward(next_state).detach().max(1)[0]
        targets_Q = reward + (self.gamma * targets_next_Q * (1 - done))

        local_Q = self.local_q_network.forward(state)
        expected_Q = local_Q.gather(1, action.unsqueeze(1).long()).squeeze(1)

        loss = F.mse_loss(expected_Q, targets_Q)
        self.local_q_network.backprop(loss)

        update_params(self.local_q_network.model, self.target_q_network.model, self.tau)

    def _loop(self, episode) -> int:
        total_reward = 0
        state = self.env.reset()
        for i in range(1000):
            action = self.policy(state)
            _state, reward, done, _ = self.env.step(action)
            self.experience_replay.remember(state, action, reward, _state, done)
            
            self.num_step = (self.num_step + 1) % 4
            if self.num_step == 0:
                experiences = self.experience_replay.recall(batch_size=64)
                self.learn(*experiences)
            state = _state
            
            total_reward += reward
            if done: break
        self.epsilon = max(self.epsilon_end, self.epsilon_decay*self.epsilon)
        return total_reward
        
try: env.close()
except: pass
env = gym.make('LunarLander-v2')
dqn = DQN(env, 
      num_episodes=2000,
      alpha=5e-4, 
      gamma=.99,
      tau=1e-3)
dqn.log_average(100)
dqn.train()

/Users/epinyoanun/miniconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Episode 100	Average Score: -220.75
Episode 200	Average Score: -192.40
Episode 300	Average Score: -108.18
Episode 400	Average Score: -66.337
Episode 500	Average Score: -81.964
Episode 600	Average Score: -42.04
Episode 700	Average Score: -62.85
Episode 800	Average Score: -72.63
Episode 889	Average Score: -63.78

KeyboardInterrupt: 

In [21]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate 
UPDATE_EVERY = 4        # how often to update the network

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, seed):
        self.my_dqn = DQN(env, 
                          num_episodes=2000,
                          alpha=5e-4, 
                          gamma=.99,
                          tau=1e-3)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done):
        self.my_dqn.experience_replay.remember(state, action, reward, next_state, done)
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
            experiences = self.my_dqn.experience_replay.recall(batch_size=64)
            self.learn(experiences, GAMMA)

    def act(self, state, eps=0.):
        return self.my_dqn.local_q_network.epsilon_greedy(state, eps)

    def learn(self, experiences, gamma):
        self.my_dqn.learn(*experiences)


import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
env = gym.make('LunarLander-v2')
env.seed(0)
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space.n)


my_dqn = DQN(env, 
              num_episodes=2000,
              alpha=5e-4, 
              gamma=.99,
              tau=1e-3)

def dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start       
    t_step = 0
    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
#         for t in range(max_t):
#             action = my_dqn.local_q_network.epsilon_greedy(state, eps)
#             next_state, reward, done, _ = env.step(action)
            
#             my_dqn.experience_replay.remember(state, action, reward, next_state, done)
#             t_step = (t_step + 1) % UPDATE_EVERY
#             if t_step == 0:
#                 experiences = my_dqn.experience_replay.recall(batch_size=64)
#                 my_dqn.learn(*experiences)
            
#             state = next_state
#             score += reward
#             if done:
#                 break 
        total_reward = 0
        state = self.env.reset()
        for i in range(1000):
            action = my_dqn.local_q_network.epsilon_greedy(state, eps)
            _state, reward, done, _ = env.step(action)
            my_dqn.experience_replay.remember(state, action, reward, _state, done)
            
            t_step = (t_step + 1) % UPDATE_EVERY
            if t_step == 0:
                experiences = my_dqn.experience_replay.recall(batch_size=64)
                my_dqn.learn(*experiences)
            state = _state
            
            score += reward
            if done: break
        self.epsilon = max(self.epsilon_end, self.epsilon_decay*self.epsilon)
        return total_reward
    
    
    
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
    return scores

agent = Agent(state_size=8, action_size=4, seed=0)
scores = dqn()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

/Users/epinyoanun/miniconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
State shape:  (8,)
Number of actions:  4
Episode 100	Average Score: -268.03
Episode 200	Average Score: -437.32
Episode 300	Average Score: -552.11
Episode 302	Average Score: -549.91

KeyboardInterrupt: 